In [2]:
import requests
import torch
import torch.nn as nn
import os
from torchvision import models
from PIL import Image

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from tqdm import tqdm

In [5]:
!pip install onnxruntime
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 52.6 MB/s eta 0:00:00


In [6]:
from torch.utils.data import Dataset
import requests
import torch
import torch.nn as nn
# Do install:
# conda install onnx
# conda install onnxruntime
import onnxruntime as ort
import numpy as np
import json
import io
import sys
import base64
from typing import Tuple
import pickle
import os
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset

In [7]:
os.makedirs("out/models", exist_ok=True)

In [8]:
model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.weight.shape[1], 10)
torch.save(model.state_dict(), "out/models/dummy_submission.pt")

### Load data

In [9]:
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [10]:
class TaskDataset(Dataset):
    def __init__(self, transform=transform):

        self.ids = []
        self.imgs = []
        self.labels = []

        # self.transform = transform

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        # img = self.transform(img)
        label = self.labels[index]
        return id_, img, label

    def __len__(self):
        return len(self.ids)

In [11]:
dataset = TaskDataset()

In [12]:
dataset: TaskDataset = torch.load("/content/drive/MyDrive/Train.pt",transform)

In [13]:
dataset

In [14]:
len(dataset)

100000

In [15]:
dataset[0]

(79431, <PIL.Image.Image image mode=RGB size=32x32>, 8)

In [16]:
images = []
ids = []
labels = []
for id, img, label in dataset:
    img = img.convert('RGB')
    images.append(img)
    ids.append(id)
    labels.append(label)


In [17]:
imgs_tensor = [transform(img) for img in images]

In [18]:
class TensorDataset(Dataset):
    def __init__(self, ids, imgs_tensor, labels):

        self.ids = ids
        self.imgs = imgs_tensor
        self.labels = labels

        # self.transform = transform

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        # img = self.transform(img)
        label = self.labels[index]
        return id_, img, label

    def __len__(self):
        return len(self.ids)

In [19]:
dataset = TensorDataset(ids, imgs_tensor, labels)


### Training

Fast Gradient Sign Method (FGSM)-generate adversarial
examples with a single gradient step

� = � + � sgn(∇!� �, �, � )

Projected Gradient Descent (PGD) - multiple random restarts.

1. Train normal model

2. Add adversarial samples during training


 maximize the loss using � and �
 ���
+∈%
� � + �, �, �

3. Minimize loss on adversarial samples


epsilon 0.01 - 0.03


Adaptive attack with more accurate gradient

compute the gradients many times, usually 10, in the forward & backward
passes, accumulate them, here in the grad_noise, and at the end average them to get a more accurate
gradient.


more layers - better. e.g. 40 if millions of params



### Evaluation

In [ ]:
#### Tests ####
# (these are being ran on the eval endpoint for every submission)

allowed_models = {
    "resnet18": models.resnet18,
    "resnet34": models.resnet34,
    "resnet50": models.resnet50,
}
with open("out/models/dummy_submission.pt", "rb") as f:
    try:
        model: torch.nn.Module = allowed_models["resnet18"](weights=None)
        model.fc = torch.nn.Linear(model.fc.weight.shape[1], 10)
    except Exception as e:
        raise Exception(
            f"Invalid model class, {e=}, only {allowed_models.keys()} are allowed",
        )
    try:
        state_dict = torch.load(f, map_location=torch.device("cpu"))
        model.load_state_dict(state_dict, strict=True)
        model.eval()
        out = model(torch.randn(1, 3, 32, 32))
    except Exception as e:
        raise Exception(f"Invalid model, {e=}")

    assert out.shape == (1, 10), "Invalid output shape"


# Send the model to the server
response = requests.post("http://34.71.138.79:9090/robustness", files={"file": open("out/models/dummy_submission.pt", "rb")}, headers={"token": "TOKEN", "model-name": "resnet18"})

# Should be 400, the clean accuracy is too low
print(response.json())

### Training Resnet

In [20]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split

In [21]:
# Assume your dataset has 1000 samples
dataset_size = len(dataset)
train_size = int(1.0 * dataset_size)  # 80% for training
test_size = dataset_size - train_size  # 20% for testing

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [22]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [23]:
import torch
import torchvision.models as models


In [24]:
model = models.resnet18(pretrained=False)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [25]:
# Modify the final fully connected layer to match your number of classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

In [26]:
import torch.optim as optim
import torch.nn.functional as F

In [27]:
# what device is avaialable for torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:

print(device)
model = model.to(device)

cuda


In [29]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [30]:
# Training loop
num_epochs = 10

In [31]:
def save_checkpoint(model, optimizer, epoch, filepath):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint, filepath)

In [32]:
# Training loop
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in tqdm(train_loader):
            ids, inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
        save_checkpoint(model, optimizer, epoch, f'model_checkpoint_epoch_{epoch}.pth')


In [ ]:
# Example usage:
train_model(model, train_loader, criterion, optimizer, num_epochs=100)

100%|██████████| 3125/3125 [00:43<00:00, 71.21it/s]


Epoch [1/100], Loss: 0.0440


100%|██████████| 3125/3125 [00:42<00:00, 73.02it/s]


Epoch [2/100], Loss: 0.0395


100%|██████████| 3125/3125 [00:43<00:00, 72.16it/s]


...

100%|██████████| 3125/3125 [00:43<00:00, 71.65it/s]


Epoch [27/100], Loss: 0.0089


100%|██████████| 3125/3125 [00:43<00:00, 71.63it/s]


Epoch [28/100], Loss: 0.0080


 95%|█████████▍| 2965/3125 [00:42<00:02, 60.11it/s]

In [ ]:
label

9

In [ ]:
def pgd_attack(model, images, labels, eps=0.03, alpha=0.01, iters=20):
    original_images = images.clone()
    for _ in range(iters):
        # print(images.dtype)
        images.requires_grad = True
        outputs = model(images)
        model.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        adv_images = images + alpha * images.grad.sign()
        eta = torch.clamp(adv_images - original_images, min=-eps, max=eps)
        images = torch.clamp(original_images + eta, min=0, max=1).detach_()
    return images

In [ ]:
# Training loop
def train_adv_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in tqdm(train_loader):
            ids, inputs, labels = batch
            #print(inputs.shape)
            inputs, labels = inputs.to(device), labels.to(device)
            adversarial_img = pgd_attack(model, inputs, labels)
            adversarial_img, labels = adversarial_img.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(adversarial_img)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
        save_checkpoint(model, optimizer, epoch, f'model_checkpoint_adversarial_epoch_{epoch}.pth')


In [ ]:
train_adv_model(model, train_loader, criterion, optimizer, num_epochs=1)

 83%|████████▎ | 2065/2500 [07:28<01:30,  4.81it/s]

In [ ]:
save_checkpoint(model, optimizer, 1, f'model_checkpoint_adversarial_epoch_0.3.pth')

In [ ]:
model = model.to("cpu")

In [ ]:
#### SUBMISSION ####
torch.save(model.state_dict(), "out/models/submission_1.pt")

#### Tests ####
# (these are being ran on the eval endpoint for every submission)

allowed_models = {
    "resnet18": models.resnet18,
    "resnet34": models.resnet34,
    "resnet50": models.resnet50,
}
with open("out/models/submission_1.pt", "rb") as f:
    # try:
    #     pass
    #     #model: torch.nn.Module = allowed_models["resnet18"](weights=None)
    #     #model.fc = torch.nn.Linear(model.fc.weight.shape[1], 10)
    # except Exception as e:
    #     raise Exception(
    #         f"Invalid model class, {e=}, only {allowed_models.keys()} are allowed",
    #     )
    try:
        state_dict = torch.load(f, map_location=torch.device("cpu"))
        model.load_state_dict(state_dict, strict=True)
        model.eval()
        out = model(torch.randn(1, 3, 32, 32))
    except Exception as e:
        raise Exception(f"Invalid model, {e=}")

    assert out.shape == (1, 10), "Invalid output shape"

In [ ]:
# Send the model to the server
response = requests.post("http://34.71.138.79:9090/robustness", files={"file": open("out/models/submission_1.pt", "rb")}, headers={"token": "75184352", "model-name": "resnet18"})

# Should be 400, the clean accuracy is too low
print(response.json())

{'detail': "Invalid model, e=RuntimeError('Error(s) in loading state_dict for ResNet:\\n\\tsize mismatch for conv1.weight: copying a param with shape torch.Size([64, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([64, 3, 7, 7]).')"}
